In [1]:
from hail import *
hc = HailContext(tmp_dir = '/nvme/tmp/shusson')

Running on Apache Spark version 2.1.0
SparkUI available at http://129.94.72.55:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.1-c72c76d


In [2]:
vds = hc.read('/nvme/tmp/shusson/sample.vds')

In [3]:
vds.summarize()

Summary(samples=0, variants=7432, call_rate=nan, contigs=['X', '12', '8', '19', '4', '15', '11', '9', 'Y', '22', '13', '16', '5', '10', '21', '6', '1', '17', '14', '20', '2', '18', '7', '3'], multiallelics=623, snps=6702, mnps=0, insertions=828, deletions=1008, complex=0, star=339, max_alleles=35)

In [18]:
split = vds.split_multi().annotate_variants_expr('va.allele = va.alleles[va.aIndex-1]')

In [48]:
kt=split.variants_table()

kt = kt.annotate('VARIANT=v, \
CHROMOSOME=v.contig, \
START=v.start, \
REF=v.ref, \
ALT=v.alt, \
RSID=va.allele.annotation.rsid, \
AC=va.allele.metrics.allele_counts.total, \
AF=if (isMissing(va.allele.metrics.allele_frequencies.total)) "\\\N" else str(va.allele.metrics.allele_frequencies.total),\
nHomRef=va.genotypes.counts.total[0], \
nHet=va.genotypes.counts.total[1], \
nHomVar=va.genotypes.counts.total[2], \
TYPE=v.altAllele.category(), \
CATO=va.allele.annotation.predictions.CATO.score, \
eigen=va.allele.annotation.predictions.Eigen.EigenRaw, \
tgpAF=va.allele.annotation.freqs.tgp.AF, \
hrcAF=va.allele.annotation.freqs.hrc.AF, \
gnomadAF=va.allele.annotation.freqs.gnomad.AF, \
feature=va.allele.annotation.vep.consequences.feature.id, \
consequences=va.allele.annotation.vep.consequences.consequences.mkString(","), \
gene=va.allele.annotation.vep.cross_references.Ensembl_gene, \
clinvar=va.allele.annotation.clinvar.ClinicalSignificance.mkString(",")')

kt = kt.select([u'VARIANT', u'CHROMOSOME', u'START', u'REF', u'ALT', u'RSID', u'AC', u'AF', u'nHomRef', u'nHet', u'nHomVar', u'TYPE', 'CATO', 'eigen', 'tgpAF', 'hrcAF', 'gnomadAF', 'feature', 'consequences', 'gene','clinvar'])


In [49]:
kt.export("sample.tsv")

2017-10-09 16:55:33 Hail: INFO: while writing:
    sample.tsv
  merge time: 10.381ms


In [30]:
kt.show()

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+---------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+--------------+-------------+-------------+---------+------+---------+----------+---------+
| VARIANT                                                                                                                                                                               | CHROMOSOME |   START | REF                                                                                                                                                                       | ALT    | RSID         |          AC | AF          | nHomRef | nHet | nHomVar | TYPE     | clinvar |
+-------------------------------------